In [ ]:
!pip install Flask

from collections import defaultdict
from flask import Flask, request, jsonify, Response
import pandas as pd
import numpy as np
import pickle
import json
import os

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install pyngrok
!ngrok authtoken 2Xiq3njrFD1bBTy6pCGqHjSxUaU_6V8fJioQysEkv6jmc7q9N
from pyngrok import ngrok
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

In [ ]:
from flask import Flask, request


def initiate():

  #@title <h1>Step1: Setup Wav2Lip</h1>
  #@markdown * Install dependency
  #@markdown * Download pretrained model
  from IPython.display import HTML, clear_output
  !rm -rf /content/sample_data
  !mkdir /content/sample_data

  !git clone https://github.com/justinjohn0306/Wav2Lip

  %cd /content/Wav2Lip

  #download the pretrained model
  !wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip.pth' -O 'checkpoints/wav2lip.pth'
  !wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth' -O 'checkpoints/wav2lip_gan.pth'
  !wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/resnet50.pth' -O 'checkpoints/resnet50.pth'
  !wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth' -O 'checkpoints/mobilenet.pth'
  a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl
  !pip install git+https://github.com/elliottzheng/batch-face.git@master

  !pip install ffmpeg-python mediapipe==0.8.11

  #this code for recording audio
  """
  To write this piece of code I took inspiration/code from a lot of places.
  It was late night, so I'm not sure how much I created or just copied o.O
  Here are some of the possible references:
  https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
  https://stackoverflow.com/a/18650249
  https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
  https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
  https://stackoverflow.com/a/49019356
  """
  from IPython.display import HTML, Audio
  from google.colab.output import eval_js
  from base64 import b64decode
  import numpy as np
  from scipy.io.wavfile import read as wav_read
  import io
  import ffmpeg

  AUDIO_HTML = """
  <script>
  var my_div = document.createElement("DIV");
  var my_p = document.createElement("P");
  var my_btn = document.createElement("BUTTON");
  var t = document.createTextNode("Press to start recording");

  my_btn.appendChild(t);
  //my_p.appendChild(my_btn);
  my_div.appendChild(my_btn);
  document.body.appendChild(my_div);

  var base64data = 0;
  var reader;
  var recorder, gumStream;
  var recordButton = my_btn;

  var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
      //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
      mimeType : 'audio/webm;codecs=opus'
      //mimeType : 'audio/webm;codecs=pcm'
    };
    //recorder = new MediaRecorder(stream, options);
    recorder = new MediaRecorder(stream);
    recorder.ondataavailable = function(e) {
      var url = URL.createObjectURL(e.data);
      var preview = document.createElement('audio');
      preview.controls = true;
      preview.src = url;
      document.body.appendChild(preview);

      reader = new FileReader();
      reader.readAsDataURL(e.data);
      reader.onloadend = function() {
        base64data = reader.result;
        //console.log("Inside FileReader:" + base64data);
      }
    };
    recorder.start();
    };

  recordButton.innerText = "Recording... press to stop";

  navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


  function toggleRecording() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... pls wait!"
    }
  }

  // https://stackoverflow.com/a/951057
  function sleep(ms) {
    return new Promise(resolve => setTimeout(resolve, ms));
  }

  var data = new Promise(resolve=>{
  //recordButton.addEventListener("click", toggleRecording);
  recordButton.onclick = ()=>{
  toggleRecording()

  sleep(2000).then(() => {
    // wait 2000ms for the data to be available...
    // ideally this should use something like await...
    //console.log("Inside data:" + base64data)
    resolve(base64data.toString())

  });

  }
  });

  </script>
  """

  %cd /
  from ghc.l_ghc_cf import l_ghc_cf
  %cd content

  def get_audio():
    display(HTML(AUDIO_HTML))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])

    process = (ffmpeg
      .input('pipe:0')
      .output('pipe:1', format='wav')
      .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, err = process.communicate(input=binary)

    riff_chunk_size = len(output) - 8
    # Break up the chunk size into four bytes, held in b.
    q = riff_chunk_size
    b = []
    for i in range(4):
        q, r = divmod(q, 256)
        b.append(r)

    # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
    riff = output[:4] + bytes(b) + output[8:]

    sr, audio = wav_read(io.BytesIO(riff))

    return audio, sr


  from IPython.display import HTML
  from base64 import b64encode
  def showVideo(path):
    mp4 = open(str(path),'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=700 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

  from IPython.display import clear_output

  clear_output()
  print("All set and ready!")

In [ ]:
# Install yt-dlp
import os

from flask import Flask, request,send_file

@app.route('/upload', methods=['POST'])
def upload_files():
    print("runninggg1")
    # Get the uploaded files from the request
    video_file = request.files['video']
    audio_file = request.files['audio']
    print("runninggg1")

    if video_file and audio_file:
        !rm -f input_vid.mp4
        !rm -f input_audio.wav
        # Save the files to a directory
        # video_file.save('input_vid.mp4')
        # audio_file.save('input_audio.wav')
        video_file.save('input_vid.mp4')
        audio_file.save('input_audio.wav')
        # Process the uploaded files as needed
        initiate()

    print("runninggg1")

    %cd /content/Wav2Lip

    # Set up paths and variables for the output file
    output_file_path = '/result_voice.mp4'

    # Delete existing output file before processing, if any
    # if os.path.exists(output_file_path):
    #     os.remove(output_file_path)
    print("runninggg1")

    pad_top =  0
    pad_bottom =  10
    pad_left =  0
    pad_right =  0
    rescaleFactor =  1
    nosmooth = True

    use_hd_model = False
    checkpoint_path = 'checkpoints/wav2lip.pth' if not use_hd_model else 'checkpoints/wav2lip_gan.pth'

    print("runninggg1")

    if nosmooth == False:
      !python inference.py --checkpoint_path $checkpoint_path --face "../input_vid.mp4" --audio "../input_audio.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor
    else:
      !python inference.py --checkpoint_path $checkpoint_path --face "../input_vid.mp4" --audio "../input_audio.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor --nosmooth
    print("runninggg1")

    return send_file(output_file_path, as_attachment=True)




In [ ]:
`
``if __name__ == '__main__':
    app.run()

NameError: ignored